# Módulos

In [1]:
import pandas as pd # Tabelas

# Requests
import requests 
from selenium import webdriver 
from time import sleep

from bs4 import BeautifulSoup as BS # Interpretação de HTMLs
from multiprocessing import Pool, cpu_count as CPUs # Paralelização

from pathlib import Path # Organização de pastas no seu computador
(Path('bases/sitemaps')).mkdir(parents = True, exist_ok = True)
import sqlite3 # Arquivamento dinâmico

# Listagem de sitemaps diários

In [2]:
portais = [
    
    'pipelinevalor',
    'umsoplaneta',
    'globorural',
    'valorinveste',
    'revistapegn',
    'epocanegocios',
    'valor'

]

sitemaps = []
for portal in portais:
    subportal = portal if portal != 'revistapegn' else 'pegn'
    pergaminho = f'https://{portal}.globo.com/sitemap/{subportal}/sitemap.xml'
    content = requests.get(pergaminho).content
    soup = BS(content, 'xml')
    sitemaps.append(list(map(lambda loc: loc.text, soup.find_all('loc'))))

# Leitura dos sitemaps diários

In [ ]:
for portal, sitemap in zip(portais, sitemaps):

    # Arquivo
    conn = sqlite3.connect(f'bases/sitemaps/sitemaps {portal}.db')
    cursor = conn.cursor()
    
    cursor.execute(f"""
    CREATE TABLE IF NOT EXISTS sitemaps (
        sitemap, 
        data_da_busca,
        url 
    );
    """)
    conn.commit()

    df = pd.read_sql('SELECT * FROM sitemaps', conn)
    sitemap = sorted(set(sitemap).difference(set(df.sitemap)), reverse = True)
    if len(sitemap) == 0:
        print(f'Já lido: {portal}..!')
        conn.close()
        continue

    # Webdriver 
    if portal == 'valor':
        driver = webdriver.Chrome()
        responses = []
        for url in sitemap:
            erros = 0
            while erros < 5:
                try:
                    driver.get(url)
                    response = driver.page_source
                    break
                except:
                    print('BOT de castigo por 1min.')
                    sleep(60) # De castigo por 1 minuto.
                    erros += 1
                    if erros > 5:
                        raise RuntimeError('Nº de tentativas excedido.')

            # Releitura fracionada
            soup = BS(response, 'xml')
            blocos = soup.find_all('url')
            conteúdo = list(map(lambda bloco: [url, 
                                               bloco.find('lastmod').text, 
                                               bloco.find('loc').text], blocos))
            df = pd.DataFrame(conteúdo, columns = ['sitemap', 'data_da_busca', 'url'])
            df.to_sql('sitemaps', conn, if_exists = 'append', index = False)
            conn.commit()
        driver.close()

    # Multirequests
    else:
        with Pool(CPUs()*3) as pool:
            responses = pool.map(requests.get, sitemap)

        # Releitura de uma vez só
        for url, response in zip(sitemap, responses):
            soup = BS(response.content, 'xml')
            blocos = soup.find_all('url')
            conteúdo = list(map(lambda bloco: [url, 
                                               bloco.find('lastmod').text,
                                               bloco.find('loc').text], blocos))
            df = pd.DataFrame(conteúdo, columns = ['sitemap', 'data_da_busca', 'url'])
            df.to_sql('sitemaps', conn, if_exists = 'append', index = False)
            conn.commit()

    print(f'Leitura concluída: {portal}..!')
    conn.close()

Leitura concluída: pipelinevalor..!
Leitura concluída: umsoplaneta..!
